In [0]:
TARGET_CATALOG="team_3d_dev.poc"

In [0]:
# Read tables from Unity Catalog
df_reviews = spark.table("samples.bakehouse.media_customer_reviews")
df_franchises = spark.table("samples.bakehouse.sales_franchises")

# Join only franchiseIDs that have reviews (inner join)
df_joined = (
    df_franchises
        .join(df_reviews, on="franchiseID", how="inner")
        .limit(100)
)

df_joined.display()

In [0]:
TARGET_TABLE=f"{TARGET_CATALOG}.franchises_with_reviews"
df_joined.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable(TARGET_TABLE)

In [0]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION {TARGET_CATALOG}.{FUNCTION_NAME}(
    num_franchises INT,
    num_results INT
)
RETURNS TABLE (
    franchiseID BIGINT,
    name STRING,
    review_count BIGINT,
    comments_array ARRAY<STRING>
)
COMMENT 'Returns franchises sorted by number of reviews in descending order.
Includes:
- review_count: total number of reviews per franchise
- comments_array: array containing all review texts per franchise
Parameters:
- num_franchises: number of distinct franchises considered
- num_results: number of rows returned.'
LANGUAGE SQL
RETURN
WITH franchise_counts AS (
    SELECT
        f.franchiseID,
        f.name,
        COUNT(r.review) AS review_count,
        COLLECT_LIST(r.review) AS comments_array
    FROM samples.bakehouse.sales_franchises f
    INNER JOIN samples.bakehouse.media_customer_reviews r
        ON f.franchiseID = r.franchiseID
    GROUP BY f.franchiseID, f.name
)
SELECT *
FROM franchise_counts
LIMIT 10
""")

## Select model playground: Meta Llama 3.3 70B Instruct
Use the tool and do a query
Example: "list the franchises with more comments, summarize the comments for me and group by franchise name"

